# T1083 - File and Directory Discovery
Adversaries may enumerate files and directories or may search in specific locations of a host or network share for certain information within a file system. Adversaries may use the information from [File and Directory Discovery](https://attack.mitre.org/techniques/T1083) during automated discovery to shape follow-on behaviors, including whether or not the adversary fully infects the target and/or attempts specific actions.

Many command shell utilities can be used to obtain this information. Examples include <code>dir</code>, <code>tree</code>, <code>ls</code>, <code>find</code>, and <code>locate</code>. (Citation: Windows Commands JPCERT) Custom tools may also be used to gather file and directory information and interact with the [Native API](https://attack.mitre.org/techniques/T1106).

## Atomic Tests

In [ ]:
#Import the Module before running the tests.
Import-Module /Users/0x6c/AtomicRedTeam/atomics/invoke-atomicredteam/Invoke-AtomicRedTeam.psd1 - Force

### Atomic Test #1 - File and Directory Discovery (cmd.exe)
Find or discover files on the file system.  Upon execution, the file "download" will be placed in the temporary folder and contain the output of
all of the data discovery commands.

**Supported Platforms:** windows
#### Attack Commands: Run with `command_prompt`
```command_prompt
dir /s c:\ >> %temp%\download
dir /s "c:\Documents and Settings" >> %temp%\download
dir /s "c:\Program Files\" >> %temp%\download
dir "%systemdrive%\Users\*.*" >> %temp%\download
dir "%userprofile%\AppData\Roaming\Microsoft\Windows\Recent\*.*" >> %temp%\download
dir "%userprofile%\Desktop\*.*" >> %temp%\download
tree /F >> %temp%\download
```

In [ ]:
Invoke-AtomicTest T1083 -TestNumbers 1

### Atomic Test #2 - File and Directory Discovery (PowerShell)
Find or discover files on the file system. Upon execution, file and folder information will be displayed.

**Supported Platforms:** windows
#### Attack Commands: Run with `powershell`
```powershell
ls -recurse
get-childitem -recurse
gci -recurse
```

In [ ]:
Invoke-AtomicTest T1083 -TestNumbers 2

### Atomic Test #3 - Nix File and Diectory Discovery
Find or discover files on the file system

References:

http://osxdaily.com/2013/01/29/list-all-files-subdirectory-contents-recursively/

https://perishablepress.com/list-files-folders-recursively-terminal/

**Supported Platforms:** macos, linux
#### Attack Commands: Run with `sh`
```sh
ls -a >> #{output_file}
if [ -d /Library/Preferences/ ]; then ls -la /Library/Preferences/ > #{output_file}; fi;
file */* *>> #{output_file}
cat #{output_file} 2>/dev/null
find . -type f
ls -R | grep ":$" | sed -e 's/:$//' -e 's/[^-][^\/]*\//--/g' -e 's/^/ /' -e 's/-/|/'
locate *
which sh
```

In [ ]:
Invoke-AtomicTest T1083 -TestNumbers 3

### Atomic Test #4 - Nix File and Directory Discovery 2
Find or discover files on the file system

**Supported Platforms:** macos, linux
#### Attack Commands: Run with `sh`
```sh
cd $HOME && find . -print | sed -e 's;[^/]*/;|__;g;s;__|; |;g' > #{output_file}
if [ -f /etc/mtab ]; then cat /etc/mtab >> #{output_file}; fi;
find . -type f -iname *.pdf >> #{output_file}
cat #{output_file}
find . -type f -name ".*"
```

In [ ]:
Invoke-AtomicTest T1083 -TestNumbers 4

## Detection
System and network discovery techniques normally occur throughout an operation as an adversary learns the environment. Data and events should not be viewed in isolation, but as part of a chain of behavior that could lead to other activities, such as Collection and Exfiltration, based on the information obtained.

Monitor processes and command-line arguments for actions that could be taken to gather system and network information. Remote access tools with built-in features may interact directly with the Windows API to gather information. Information may also be acquired through Windows system management tools such as [Windows Management Instrumentation](https://attack.mitre.org/techniques/T1047) and [PowerShell](https://attack.mitre.org/techniques/T1059/001).